### DataSet
1. 每个数据集代表一个数据来源，可以是张量、TFRecord、文本文件等
1. 从数据集中读取数据时，需要一个迭代器
1. 使用迭代器的get_next()方法读取数据
1. 类似TFRecord有各自不同的feature格式，因此需要提供一个parser函数来解析所读取的TFRecord数据
1. dataset.map(parser)表示对数据集中的每一条数据都调用parser函数进行加工处理
1. dataset也提供相应的批量处理方法    
    dataset.shuffle(buffer_size) #buffer_size 类似tf.train.shuffle_batch的min_after_dequeue    
    dataset.batch(batch_size)    #batch_size 每个batch有多少数据    
1. dataset.repeat(N) #将数据集重复N份，每份称为一个epoch； 如果repeat前已经进行过shuffle，则个epoch数据都不一样； 

1.从 Numpy 导入
```
x = np.random.sample((100,2))
# make a dataset from a numpy array
dataset = tf.data.Dataset.from_tensor_slices(x)

features, labels = (np.random.sample((100,2)), np.random.sample((100,1)))
dataset = tf.data.Dataset.from_tensor_slices((features,labels))
```
1. 从张量导入
```
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([100, 2]))
```
1. 从placeholder导入
```
x = tf.placeholder(tf.float32, shape=[None,2])
dataset = tf.data.Dataset.from_tensor_slices(x)
```
1. 从生成器导入    
```
sequence = np.array([[1],[2,3],[3,4]])
def generator():
    for el in sequence:
        yield el
      
dataset = tf.data.Dataset().from_generator(generator,
                                           output_types=tf.float32, 
                                           output_shapes=[tf.float32])
```

#### 1. 从数组创建数据集。

In [2]:
import tempfile
import tensorflow as tf

input_data = [1, 2, 3, 5, 8]
dataset = tf.data.Dataset.from_tensor_slices(input_data)

# 定义迭代器。
iterator = dataset.make_one_shot_iterator()

# get_next() 返回代表一个输入数据的张量。
x = iterator.get_next()
y = x * x

with tf.Session() as sess:
    for i in range(len(input_data)):
        print(sess.run(y))


1
4
9
25
64


#### 2. 读取文本文件里的数据。

In [3]:
# 创建文本文件作为本例的输入。
with open("./test1.txt", "w") as file:
    file.write("File1, line1.\n") 
    file.write("File1, line2.\n")
with open("./test2.txt", "w") as file:
    file.write("File2, line1.\n") 
    file.write("File2, line2.\n")

# 从文本文件创建数据集。这里可以提供多个文件。
input_files = ["./test1.txt", "./test2.txt"]
dataset = tf.data.TextLineDataset(input_files)

# 定义迭代器。
iterator = dataset.make_one_shot_iterator()

# 这里get_next()返回一个字符串类型的张量，代表文件中的一行。
x = iterator.get_next()  
with tf.Session() as sess:
    for i in range(4):
        print(sess.run(x))


File1, line1.
File1, line2.
File2, line1.
File2, line2.


#### 3. 解析TFRecord文件里的数据。读取文件为本章第一节创建的文件。

In [4]:
# 解析一个TFRecord的方法。
def parser(record):
    features = tf.parse_single_example(
        record,
        features={
            'image_raw':tf.FixedLenFeature([],tf.string),
            'pixels':tf.FixedLenFeature([],tf.int64),
            'label':tf.FixedLenFeature([],tf.int64)
        })
    decoded_images = tf.decode_raw(features['image_raw'],tf.uint8)
    retyped_images = tf.cast(decoded_images, tf.float32)
    images = tf.reshape(retyped_images, [784])
    labels = tf.cast(features['label'],tf.int32)
    #pixels = tf.cast(features['pixels'],tf.int32)
    return images, labels

# 从TFRecord文件创建数据集。这里可以提供多个文件。
input_files = ["output.tfrecords"]
dataset = tf.data.TFRecordDataset(input_files)

# map()函数表示对数据集中的每一条数据进行调用解析方法。
dataset = dataset.map(parser)

# 定义遍历数据集的迭代器。
iterator = dataset.make_one_shot_iterator()

# 读取数据，可用于进一步计算
image, label = iterator.get_next()

with tf.Session() as sess:
    for i in range(10):
        x, y = sess.run([image, label]) 
        print(y)


7
3
4
6
1
8
1
0
9
8


#### 4. 使用initializable_iterator来动态初始化数据集。

In [5]:
# 从TFRecord文件创建数据集，具体文件路径是一个placeholder，稍后再提供具体路径。
input_files = tf.placeholder(tf.string)
dataset = tf.data.TFRecordDataset(input_files)
dataset = dataset.map(parser)

# 定义遍历dataset的initializable_iterator。
iterator = dataset.make_initializable_iterator()
image, label = iterator.get_next()

with tf.Session() as sess:
    # 首先初始化iterator，并给出input_files的值。
    sess.run(iterator.initializer,
             feed_dict={input_files: ["output.tfrecords"]})
    # 遍历所有数据一个epoch。当遍历结束时，程序会抛出OutOfRangeError。
    while True:
        try:
            x, y = sess.run([image, label])
        except tf.errors.OutOfRangeError:
            break 
